In [3]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score

In [7]:
train=pd.read_csv(r'C:\atlantis_citizens_final.csv')
test=pd.read_csv(r'C:\test_atlantis_hidden.csv')

In [8]:
train

,Citizen_ID,Diet_Type,District_Name,Occupation,Wealth_Index,House_Size_sq_ft,Life_Expectancy,Vehicle_Owned,Work_District,Bio_Hash
0,CIT_15935,Exotic Imports,Coral Slums,Scribe,1491.0,100.0,42.0,Fin Bicycle,Mariana Plaza,b81cb8ce
1,CIT_11623,Seafood,Coral Slums,Fisher,1596.0,100.0,49.0,Sea Scooter,Deep Trench,72f48eef
2,CIT_8026,Seafood,Mariana Plaza,Warrior,3921.0,533.0,37.0,Sea Scooter,Deep Trench,0abde296
3,CIT_0492,Exotic Imports,Deep Trench,Fisher,NaN,136.0,38.0,Fin Bicycle,Deep Trench,8055fc9e
4,CIT_0275,Seaweed,Deep Trench,Warrior,25985.0,2673.0,54.0,Sea Scooter,Deep Trench,c77829e2
...,...,...,...,...,...,...,...,...,...,...
15746,CIT_10602,Seaweed,Mariana Plaza,Merchant,2896.0,428.0,59.0,Sea Scooter,Mariana Plaza,268659a4
15747,CIT_0865,Seaweed,Coral Slums,Miner,1671.0,100.0,47.0,No Vehicle,Mariana Plaza,ca287a73
15748,CIT_0809,Exotic Imports,Mariana Plaza,Scribe,4656.0,756.0,53.0,Fin Bicycle,Mariana Plaza,717ca395
15749,CIT_10750,Seafood,The Golden Reef,Merchant,17529.0,3961.0,72.0,No Vehicle,The Golden Reef,34fb8c73


In [9]:
test

,Citizen_ID,Diet_Type,District_Name,Wealth_Index,House_Size_sq_ft,Life_Expectancy,Vehicle_Owned,Work_District,Bio_Hash
0,CIT_15383,Seaweed,Mariana Plaza,2851,453,53,Sea Scooter,Coral Slums,0392e7e1
1,CIT_14830,Seaweed,Mariana Plaza,5176,815,58,No Vehicle,Deep Trench,da47b527
2,CIT_17388,Exotic Imports,Deep Trench,3772,390,29,Fin Bicycle,The Golden Reef,970f8714
3,CIT_17438,Seaweed,Deep Trench,1288,131,44,No Vehicle,Deep Trench,7ca70526
4,CIT_16735,Seafood,Deep Trench,1736,182,43,Sea Scooter,The Golden Reef,c85a5d8b
...,...,...,...,...,...,...,...,...,...
3933,CIT_15659,Exotic Imports,Mariana Plaza,3537,572,50,Fin Bicycle,Mariana Plaza,8b662009
3934,CIT_16061,Seafood,Coral Slums,1769,116,41,Sea Scooter,Deep Trench,7d7f979a
3935,CIT_17913,Seaweed,The Golden Reef,7972,1689,72,No Vehicle,Coral Slums,f182c545
3936,CIT_17666,Exotic Imports,Coral Slums,1904,138,34,Sea Scooter,Coral Slums,9ab7b011


In [10]:
y=train['Occupation']

In [13]:
X=train.drop(columns=['Occupation'])

In [26]:
categorical_columns=X.select_dtypes(include="object").columns
numerical_columns=X.select_dtypes(exclude="object").columns

In [16]:
category_columns=category_columns.drop("Citizen_ID",errors="ignore")
numerical_columns=numerical_columns.drop("Citizen_ID",errors="ignore")

In [23]:
numerical_pipeline=Pipeline([("imputer",SimpleImputer(strategy="median"))])

In [24]:
categorical_pipeline=Pipeline([("imputer",SimpleImputer(strategy="most_frequent")),("encoder",OneHotEncoder(handle_unknown="ignore"))])

In [28]:
preprocess=ColumnTransformer([("num",num_pipeline,numerical_columns),("cat",cat_pipeline,categorical_columns)])

In [30]:
model=GradientBoostingClassifier(n_estimators=300,learning_rate=0.05,max_depth=3,random_state=42)

In [33]:
pipe=Pipeline([("preprocess",preprocess),("model",model)])

In [34]:
cv=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

In [35]:
scores=cross_val_score(pipe,X,y,cv=cv,scoring="f1_macro")

In [36]:
print("CV scores:",scores)
print("Mean Macro F1:",scores.mean())

CV scores: [0.602962   0.60459907 0.60463544 0.60550785 0.6226496 ]
Mean Macro F1: 0.6080707908369296


In [37]:
pipe.fit(X,y)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [50]:
test_features=test.copy()

In [51]:
test_preds=pipe.predict(test_features)

In [55]:
submission=pd.DataFrame({"Citizen_ID":test["Citizen_ID"],"Occupation":test_preds})

In [56]:
submission.to_csv("submission7.csv",index=False)

In [57]:
submission

,Citizen_ID,Occupation
0,CIT_15383,Miner
1,CIT_14830,Scribe
2,CIT_17388,Warrior
3,CIT_17438,Fisher
4,CIT_16735,Fisher
...,...,...
3933,CIT_15659,Scribe
3934,CIT_16061,Fisher
3935,CIT_17913,Merchant
3936,CIT_17666,Miner


In [58]:
occupation_map = {
    "Warrior": 0,
    "Merchant": 1,
    "Fisher": 2,
    "Miner": 3,
    "Scribe": 4
}

In [59]:
df=pd.read_csv(r'submission7.csv')

In [60]:
df["Occupation"] = df["Occupation"].map(occupation_map)

In [61]:
df["Occupation"].unique()

array([3, 4, 0, 2, 1])

In [63]:
df.to_csv("submission_fixed2.csv", index=False)